In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
!pip install sentencepiece transformers datasets

In [ ]:
import pandas as pd

with open("Dataset 2 (Hindi - Hinglish)/test_devanagari.txt", "r") as f:
    lines = f.read()
    
lines_array = lines.split("\n")
lines_array.pop()
test_split = [line.split("\t") for line in lines_array]

with open("Dataset 2 (Hindi - Hinglish)/train_devanagari.txt", "r") as f:
    lines = f.read()
    
lines_array = lines.split("\n")
lines_array.pop()
train_split = [line.split("\t") for line in lines_array]
    
with open("Dataset 2 (Hindi - Hinglish)/dev_devanagari.txt", "r") as f:
    lines = f.read()

lines_array = lines.split("\n")
lines_array.pop()
dev_split = [line.split("\t") for line in lines_array]

df = {}
df["test"] = pd.DataFrame(test_split, columns=["input", "output"])
df["train"] = pd.DataFrame(train_split, columns=["input", "output"])
df["dev"] = pd.DataFrame(dev_split, columns=["input", "output"])

from datasets import load_dataset, Dataset, DatasetDict

ds = DatasetDict()
ds["test"] = Dataset.from_pandas(df["test"])
ds["train"] = Dataset.from_pandas(df["train"])
ds["dev"] = Dataset.from_pandas(df["dev"])


In [ ]:
ds

In [ ]:
def tokenise_ds(example):
    return tokenizer.prepare_seq2seq_batch(src_texts=example["input"], tgt_texts=example["output"])

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")
tokenizer.model_max_length = 64

def tokenise_ds(example):
    return tokenizer.prepare_seq2seq_batch(src_texts=example["input"], tgt_texts=example["output"])

    
tokenized_datasets = ds.map(tokenise_ds, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding, AutoModelForSeq2SeqLM, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")
# model = BertForSequenceClassification.from_pretrained("ai4bharat/indic-bert")
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model=model)


In [ ]:
from transformers import Trainer

from transformers import TrainingArguments

checkpoint_path = "./results/checkpoint-29500"

training_args = TrainingArguments(
    output_dir="./results-Part2",
    num_train_epochs=15,
    per_device_train_batch_size=16,
    learning_rate=0.001,
    resume_from_checkpoint=checkpoint_path,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

# Test Scores

In [ ]:
checkpoint_path = "./results/checkpoint-10000"
from transformers import DataCollatorWithPadding, AutoModelForSeq2SeqLM, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq
device = "cuda:0"

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
model = model.to(device)
# model = BertForSequenceClassification.from_pretrained("ai4bharat/indic-bert")
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model=model)

In [ ]:
from sacrebleu.metrics import BLEU
import sacrebleu

# Test Score

In [ ]:
output_sent = []
for idx, sent in enumerate(ds["test"]["input"]):
  sent_token = tokenizer(sent, return_tensors="pt").input_ids
  output = model.generate(sent_token.cuda(), max_length=200)
  output_sent.append(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
# Define lists to store English and Hindi sentences
sys = []
refs = []
ct = 0
# Open the text file for reading
with open('Dataset 2 (Hindi - Hinglish)/test_devanagari.txt', 'r', encoding='utf-8') as file:
    for line in file:
        # Split each line into English and Hindi sentences using '\t' as the separator
        eng, hin = line.strip().split('\t')
        sys.append(output_sent[ct])
        refs.append([hin])
        ct += 1
# Close the file
file.close()

In [ ]:
bleu = BLEU()

# Calculate BLEU scores for each candidate-reference pair
bleu_scores = [sacrebleu.corpus_bleu([candidate], [reference]).score for candidate, reference in zip(sys, refs)]

# Calculate the average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {average_bleu:.2f}")


In [ ]:
# Specify the file path where you want to write the text
file_path = "testScore2.txt"  # Change this to the desired file path
# Open the file in write mode ('w')
with open(file_path, 'w') as file:
    # Write the string to the file
    file.write(str(average_bleu))

# Optionally, you can close the file explicitly, although using 'with' will automatically close it
# file.close()

print(f"The string has been written to {file_path}")

# Train

In [ ]:
output_sent = []
for idx, sent in enumerate(ds["train"]["input"]):
  sent_token = tokenizer(sent, return_tensors="pt").input_ids
  output = model.generate(sent_token.cuda(), max_length=200)
  output_sent.append(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
# Define lists to store English and Hindi sentences
sys = []
refs = []
ct = 0
# Open the text file for reading
with open('Dataset 2 (Hindi - Hinglish)/train_devanagari.txt', 'r', encoding='utf-8') as file:
    for line in file:
        # Split each line into English and Hindi sentences using '\t' as the separator
        eng, hin = line.strip().split('\t')
        sys.append(output_sent[ct])
        refs.append([hin])
        ct += 1

# Close the file
file.close()

In [ ]:
bleu = BLEU()

# Calculate BLEU scores for each candidate-reference pair
bleu_scores = [sacrebleu.corpus_bleu([candidate], [reference]).score for candidate, reference in zip(sys, refs)]

# Calculate the average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {average_bleu:.2f}")


In [ ]:
# Specify the file path where you want to write the text
file_path = "trainScore2.txt"  # Change this to the desired file path
# Open the file in write mode ('w')
with open(file_path, 'w') as file:
    # Write the string to the file
    file.write(str(average_bleu))

# Optionally, you can close the file explicitly, although using 'with' will automatically close it
# file.close()

print(f"The string has been written to {file_path}")

# Dev

In [ ]:
output_sent = []
for idx, sent in enumerate(ds["dev"]["input"]):
  sent_token = tokenizer(sent, return_tensors="pt").input_ids
  output = model.generate(sent_token.cuda(), max_length=200)
  output_sent.append(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
# Define lists to store English and Hindi sentences
sys = []
refs = []
ct = 0
# Open the text file for reading
with open('Dataset 2 (Hindi - Hinglish)/test_devanagari.txt', 'r', encoding='utf-8') as file:
    for line in file:
        # Split each line into English and Hindi sentences using '\t' as the separator
        eng, hin = line.strip().split('\t')
        sys.append(output_sent[ct])
        refs.append([hin])
        ct += 1

# Close the file
file.close()

In [ ]:
bleu = BLEU()

# Calculate BLEU scores for each candidate-reference pair
bleu_scores = [sacrebleu.corpus_bleu([candidate], [reference]).score for candidate, reference in zip(sys, refs)]

# Calculate the average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {average_bleu:.2f}")


In [ ]:
# Specify the file path where you want to write the text
file_path = "devScore.txt"  # Change this to the desired file path
# Open the file in write mode ('w')
with open(file_path, 'w') as file:
    # Write the string to the file
    file.write(str(average_bleu))

# Optionally, you can close the file explicitly, although using 'with' will automatically close it
# file.close()

print(f"The string has been written to {file_path}")